# Mapa interativo para busca de clientes no estado de Pernambuco

## Bibliotecas utilizadas no projeto


In [1]:
import pandas as pd
import geopandas as gd
import keplergl as kp

c:\Program Files\python\Lib\site-packages\keplergl\keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


## Limpeza e tratamento dos dados da receita federal

O volume de dados fornecido pela Receita Federal é elevado, porém segue um único padrão que facilitará o processo como um todo. Tendo em vista este padrão, realizaremos o tratamento de um dos conjuntos de dados e construiremos uma função para tratamento dos demais conjuntos de dados.

OBS: temos algumas informações relevantes sobre o arquivo que são necessárias ao correto carregamento dos dados no pandas, são elas:
- O arquivo é do tipo CSV, logo, utilizaremos a função read_csv do pandas;
- O separador de colunas deste arquivo é ";" e indicaremos isto com o argumento "sep='; "';
- O arquivo foi construído com unicode ANSI e indicaremos isto com o argumento "encoding='ANSI";
- O arquivo não possui títulos para as colunas e precisamos especificar esta informação pare evitar a "promoção" da primeira linha como título da coluna, para isto utilizaremos o argumento "header=None";
- Passaremos a lista de titulos "titulos_colum" para o argumento "names=" que fará a atribuição dos títulos ao dataframe.

Dadas as observações criamos duas variáveis para armazenar o caminho do arquivo e títulos das colunas 


In [2]:
#caminho do arquivo
caminho_dados = "D:/bkp_humberto/PROJETOS/Geo_int/ENTRADA/estab-01.csv"

#Nome das colunas para reatribuição
titulos_colum = [
    "cnpj_basico",
    "cnpj_ordem",
    "cnjp_dv",
    "identificador_matriz",
    "nome_fantasia",
    "situacao_cadastral",
    "data_situacao",
    "motivo_situacao",
    "nome_cidade_exterior",
    "pais",
    "data_inicio_atv",
    "CNAE_principal",
    "CNAE_secundario",
    "tipo_logradouro",
    "logradouro",
    "numero_logra",
    "complemento",
    "bairro",
    "CEP",
    "UF",
    "municipio",
    "ddd1","telefone1",
    "ddd2","telefone2",
    "ddd_fax",'fax',
    "email",
    "situacao_especial",
    "data_situacao_especial"
]

Após a construção das variáveis de apoio realizaremos a carga dos dados com seus devidos cuidados(separador, codificação,título).

In [7]:
#carga dos dados
df_RF_01 = pd.read_csv(caminho_dados,sep=';',encoding='ANSI', header=None,names=titulos_colum)

C:\Users\hcage\AppData\Local\Temp\ipykernel_19676\3529961736.py:2: DtypeWarning: Columns (8,21,22,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_RF_01 = pd.read_csv(caminho_dados,sep=';',encoding='ANSI', header=None,names=titulos_colum)


Nosso objetivo é listar apenas as empresas em situação ativa, neste sentido raelizaremos um filtro na coluna "situacao_cadastral" para todos os correspondentes ao valor "2" que conforme dicionário de dados corresponde as empresas ativas.

In [8]:
#filtro de cnpj's ativos
df_RF_ativos = df_RF_01[df_RF_01['situacao_cadastral']==2 ]
df_RF_ativos.head(2)

,cnpj_basico,cnpj_ordem,cnjp_dv,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao,motivo_situacao,nome_cidade_exterior,pais,...,municipio,ddd1,telefone1,ddd2,telefone2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
12,591451,1,40,1,QUIM-BASE,2,20051103,0,NaN,NaN,...,7107,11.0,56415044,NaN,NaN,11.0,56415044,UTILEX.LEGAL@BOL.COM.BR,NaN,NaN
16,7396980,1,32,1,NaN,2,0,0,NaN,NaN,...,6555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Faremos a remoção das colunas não interessantes ao nosso objetivo

In [3]:
#lista de colunas para remoção
colum_drop = ['situacao_cadastral','data_situacao','motivo_situacao','nome_cidade_exterior','pais','situacao_especial','data_situacao_especial']
#drop de colunas
df_RF_ativos = df_RF_ativos.drop(colum_drop, axis=1)

NameError: name 'df_RF_ativos' is not defined

Por fim, faremos um filtro de estado e salvaremos o resultado em um arquivo do tipo csv

In [23]:
#Filtro para coletar dados do estado de pernambuco
df_RF_PE = df_RF_ativos[df_RF_ativos['UF']=='PE']

#Exportando os valores salvos em um arquivo do tipo csv
df_RF_PE.to_csv('Dados_pe/pe_01')

### Funçao para limpeza dos dados

Como visto o processo de limepeza ocorreu conforme o desejado e neste ponto criaremos uma função que realize o processamento dos demais arquivos

In [4]:
def pre_limpeza(arquivo , caminho_carga ,UF, destino_salvamento , titulos_colum, colunas_drop):

    #carga dos dados e adição de títulos as colunas
    df_carga = pd.read_csv(f'{caminho_carga}/{arquivo}',sep=';',encoding='ANSI', header=None,names=titulos_colum)

    #filtro de cnpj's ativos
    df_ativos = df_carga[df_carga['situacao_cadastral']==2 ]

    #Remoção de colunas não necessárias as análise
    df_limpo = df_ativos.drop(colunas_drop, axis=1)

    # filtro de estado
    df_final = df_limpo[df_limpo['UF']==UF]

    #salvar arquivo em formato csv no destino designado com nome do arquivo acrescido por sulfixo
    df_final.to_csv(f'{destino_salvamento}/{arquivo}_tratado')

    #Função de saída meramente informativa
    return(f"arquivo limpo e salvo {arquivo}_tratado.csv")

Criada a função, criaremos variáveis auxiliáres e um laço de repetição para efetuar o tratamento dos demais arquivos brutos da Receita Federa.

In [5]:
#variáveis auxiliares a serem utilizadas

#Local do arquivo
caminho_dado = "D:/bkp_humberto/PROJETOS/Geo_int/ENTRADA"

#Lista de nomes dos arquivos
arquivos_list = [
    'estab-00.csv', 'estab-01.csv',
    'estab-02.csv', 'estab-03.csv',
    'estab-04.csv', 'estab-05.csv',
    'estab-06.csv', 'estab-07.csv',
    'estab-08.csv', 'estab-09.csv'
]

#lista de estados ou dado unitário
UF_busca = 'PE'

#Caminho de destino para os dados tratados
caminho_dado_limpo = 'Dados_pe'

In [9]:
#laço de repetição para coletar e processamento dos dados
for i in range(10):
    pre_limpeza(arquivos_list[i],caminho_dado,UF_busca,caminho_dado_limpo,titulos_colum,colum_drop)

C:\Users\hcage\AppData\Local\Temp\ipykernel_1692\810206453.py:4: DtypeWarning: Columns (8,18,21,22,23,24,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_carga = pd.read_csv(f'{caminho_carga}/{arquivo}',sep=';',encoding='ANSI', header=None,names=titulos_colum)
C:\Users\hcage\AppData\Local\Temp\ipykernel_1692\810206453.py:4: DtypeWarning: Columns (8,21,22,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_carga = pd.read_csv(f'{caminho_carga}/{arquivo}',sep=';',encoding='ANSI', header=None,names=titulos_colum)
C:\Users\hcage\AppData\Local\Temp\ipykernel_1692\810206453.py:4: DtypeWarning: Columns (8,21,22,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_carga = pd.read_csv(f'{caminho_carga}/{arquivo}',sep=';',encoding='ANSI', header=None,names=titulos_colum)
C:\Users\hcage\AppData\Local\Temp\ipykernel_1692\810206453.py:4: DtypeWarning: Columns (8,21,22,24,26,28) have mixe

Após alguns bons minutos de espera, faremos uma nova função para realizar a junção dos arquivos e posterior geocodificação. 

In [ ]:
def junta_df(lista_df):
